# Import libraries and data

In [1]:
import pandas as pd 
from mlxtend.frequent_patterns import apriori, association_rules 

In [2]:
POIdata_cityA = pd.read_csv('POIdata_cityA.csv')
POIdata_cityB = pd.read_csv('POIdata_cityB.csv')
POIdata_cityC = pd.read_csv('POIdata_cityC.csv')
POIdata_cityD = pd.read_csv('POIdata_cityD.csv')

POI_datacategories = pd.read_csv('POI_datacategories.csv', header=None)

In [3]:
POIdata_cityA.head()

,x,y,category,POI_count
0,1,1,74,4
1,1,1,48,4
2,1,1,79,2
3,1,1,69,2
4,1,1,73,1


In [4]:
POI_datacategories.head()

,0
0,Food
1,Shopping
2,Entertainment
3,Japanese restaurant
4,Western restaurant


Check for any na values

In [5]:
POIdata_cityA.isna().sum()

x            0
y            0
category     0
POI_count    0
dtype: int64

In [6]:
POIdata_cityB.isna().sum()

x            0
y            0
category     0
POI_count    0
dtype: int64

In [7]:
POIdata_cityC.isna().sum()

x            0
y            0
category     0
POI_count    0
dtype: int64

In [8]:
POIdata_cityD.isna().sum()

x            0
y            0
category     0
POI_count    0
dtype: int64

# Splitting of data

Mobility Data Files
- uid: User ID 
- d: Current date 
- t: Specific time of the record 
- x: x-axis coordinate (grid column number, counted from left to right, starting from 1) 
- y: y-axis coordinate (grid row number, counted from top to bottom, starting from 1)

POI Distribution Data
- x, y: The x and y-axis coordinates of the grid 
- category: POI category 
- POI_count: The number of that POI category present in the grid 

In [9]:
POIdata_cityA['xy'] = POIdata_cityA['x'].apply(lambda x: f"{x:03d}") + POIdata_cityA['y'].apply(lambda y: f"{y:03d}")
POIdata_cityB['xy'] = POIdata_cityB['x'].apply(lambda x: f"{x:03d}") + POIdata_cityB['y'].apply(lambda y: f"{y:03d}")
POIdata_cityC['xy'] = POIdata_cityC['x'].apply(lambda x: f"{x:03d}") + POIdata_cityC['y'].apply(lambda y: f"{y:03d}")
POIdata_cityD['xy'] = POIdata_cityD['x'].apply(lambda x: f"{x:03d}") + POIdata_cityD['y'].apply(lambda y: f"{y:03d}")

POIdata_cityA.head()

,x,y,category,POI_count,xy
0,1,1,74,4,001001
1,1,1,48,4,001001
2,1,1,79,2,001001
3,1,1,69,2,001001
4,1,1,73,1,001001


In [10]:
basket_cityA = POIdata_cityA.groupby(['xy', 'category'])['POI_count'].sum().unstack().reset_index().fillna(0).set_index(['xy'])
basket_cityB = POIdata_cityB.groupby(['xy', 'category'])['POI_count'].sum().unstack().reset_index().fillna(0).set_index(['xy'])
basket_cityC = POIdata_cityC.groupby(['xy', 'category'])['POI_count'].sum().unstack().reset_index().fillna(0).set_index(['xy'])
basket_cityD = POIdata_cityD.groupby(['xy', 'category'])['POI_count'].sum().unstack().reset_index().fillna(0).set_index(['xy'])

basket_cityA.head()

category,1,2,3,4,5,6,7,8,9,10,...,76,77,78,79,80,81,82,83,84,85
xy,,,,,,,,,,,,,,,,,,,,,
001001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
001002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
001003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0
001004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0
001005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
basket_cityA = basket_cityA.astype(bool)
basket_cityB = basket_cityB.astype(bool)
basket_cityC = basket_cityC.astype(bool)
basket_cityD = basket_cityD.astype(bool)

basket_cityA.head()

category,1,2,3,4,5,6,7,8,9,10,...,76,77,78,79,80,81,82,83,84,85
xy,,,,,,,,,,,,,,,,,,,,,
001001,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
001002,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False
001003,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,True,False,False,False
001004,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,True,False,False,False,False
001005,False,False,False,False,False,False,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False


In [12]:
category_names = {i: POI_datacategories.iloc[i-1, 0] for i in range(1, 86)}

basket_cityA.columns = [category_names[col] for col in basket_cityA.columns]
basket_cityB.columns = [category_names[col] for col in basket_cityB.columns]
basket_cityC.columns = [category_names[col] for col in basket_cityC.columns]
basket_cityD.columns = [category_names[col] for col in basket_cityD.columns]

basket_cityA.head()

,Food,Shopping,Entertainment,Japanese restaurant,Western restaurant,Eat all you can restaurant,Chinese restaurant,Indian restaurant,Ramen restaurant,Curry restaurant,...,Accountant Office,IT Office,Publisher Office,Building Material,Gardening,Heavy Industry,NPO,Utility Copany,Port,Research Facility
xy,,,,,,,,,,,,,,,,,,,,,
001001,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
001002,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False
001003,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,True,False,False,False
001004,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,True,False,False,False,False
001005,False,False,False,False,False,False,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False


# Building the models and analyzing the results

In [13]:
def apriori_algorithm(basket, min_support):

    # Building the model 
    frq_items = apriori(basket, min_support = min_support, use_colnames = True, low_memory=True) 

    # Collecting the inferred rules in a dataframe 
    rules = association_rules(frq_items, metric ="lift", min_threshold = 1, num_itemsets = basket.shape[1]) 
    rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

    return rules

In [14]:
rules_cityA = apriori_algorithm(basket_cityA, 0.06)
rules_cityA.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
3296614,"(Café, Hospital, Transit Station, Accountant O...",(Hair Salon),0.060856,0.374814,0.060111,0.987765,2.635348,1.0,0.037302,51.098547,0.660754,0.160058,0.980430,0.574071
796122,"(Café, Hospital, Clothes Store, Japanese resta...",(Hair Salon),0.063089,0.374814,0.062295,0.987411,2.634405,1.0,0.038648,49.663224,0.662184,0.165852,0.979864,0.576807
2462421,"(Café, Hospital, Transit Station, Fishing, Lau...",(Hair Salon),0.062196,0.374814,0.061402,0.987231,2.633922,1.0,0.038090,48.959890,0.661479,0.163473,0.979575,0.575525
4066489,"(Café, Hospital, Transit Station, Japanese res...",(Hair Salon),0.061451,0.374814,0.060657,0.987076,2.633510,1.0,0.037624,48.373778,0.660891,0.161491,0.979328,0.574454
788125,"(Japanese restaurant, Real Estate, Fishing, La...",(Hair Salon),0.060905,0.374814,0.060111,0.986960,2.633201,1.0,0.037283,47.943962,0.660460,0.160037,0.979142,0.573668


In [15]:
rules_cityB = apriori_algorithm(basket_cityB, 0.03)
rules_cityB.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
393488,"(Home Appliances, Accountant Office, Post Offi...",(Hair Salon),0.030688,0.1895,0.030688,1.0,5.277039,1.0,0.024873,inf,0.836160,0.161943,1.0,0.580972
393548,"(Heavy Industry, Home Appliances, Post Office,...",(Hair Salon),0.030579,0.1895,0.030579,1.0,5.277039,1.0,0.024784,inf,0.836066,0.161365,1.0,0.580682
867646,"(Café, Japanese restaurant, Heavy Industry, Re...",(Hair Salon),0.031127,0.1895,0.031127,1.0,5.277039,1.0,0.025228,inf,0.836538,0.164257,1.0,0.582128
982285,"(Cram School, Interior Shop, Accountant Office...",(Hair Salon),0.030688,0.1895,0.030688,1.0,5.277039,1.0,0.024873,inf,0.836160,0.161943,1.0,0.580972
1030769,"(Japanese restaurant, Heavy Industry, Real Est...",(Hair Salon),0.031565,0.1895,0.031565,1.0,5.277039,1.0,0.025584,inf,0.836917,0.166570,1.0,0.583285


In [16]:
rules_cityC = apriori_algorithm(basket_cityC, 0.09)
rules_cityC.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
3951142,"(Elderly Care Home, Bank, Home Appliances, Par...",(Real Estate),0.093817,0.396186,0.093817,1.000000,2.524068,1.0,0.056648,inf,0.666327,0.236801,1.000000,0.618401
4696044,"(Elderly Care Home, Bank, Home Appliances, Par...",(Real Estate),0.092279,0.396186,0.092279,1.000000,2.524068,1.0,0.055720,inf,0.665198,0.232919,1.000000,0.616460
3911771,"(Elderly Care Home, Bank, Hospital, Home Appli...",(Real Estate),0.102122,0.396186,0.101815,0.996988,2.516466,1.0,0.061355,200.466318,0.671158,0.256788,0.995012,0.626988
2846234,"(Elderly Care Home, Bank, Home Appliances, Acc...",(Real Estate),0.100277,0.396186,0.099969,0.996933,2.516326,1.0,0.060241,196.843433,0.669756,0.252133,0.994920,0.624631
4628213,"(Elderly Care Home, Bank, Hospital, Home Appli...",(Real Estate),0.100277,0.396186,0.099969,0.996933,2.516326,1.0,0.060241,196.843433,0.669756,0.252133,0.994920,0.624631


In [17]:
rules_cityD = apriori_algorithm(basket_cityD, 0.03)
rules_cityD.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
711730,"(Hospital, Transit Station, Accountant Office,...",(Hair Salon),0.032490,0.230251,0.032308,0.994398,4.318752,1.0,0.024827,137.400164,0.794257,0.140205,0.992722,0.567357
941177,"(Hospital, Transit Station, Accountant Office,...",(Hair Salon),0.030943,0.230251,0.030761,0.994118,4.317535,1.0,0.023636,130.857299,0.792922,0.133491,0.992358,0.563857
941685,"(Hospital, Transit Station, Accountant Office,...",(Hair Salon),0.031216,0.230251,0.030943,0.991254,4.305097,1.0,0.023755,88.007948,0.792454,0.134228,0.988637,0.562820
943463,"(Hospital, Transit Station, Accountant Office,...",(Hair Salon),0.030397,0.230251,0.030124,0.991018,4.304073,1.0,0.023125,85.698702,0.791728,0.130675,0.988331,0.560924
713368,"(Hospital, Transit Station, Accountant Office,...",(Hair Salon),0.032763,0.230251,0.032399,0.988889,4.294827,1.0,0.024855,69.277394,0.793148,0.140489,0.985565,0.564800


# Analysis of rule associations

## 1. Analysis of City A

In [18]:
cityA_analysis = rules_cityA.iloc[:,0:7]
cityA_analysis['interest'] = cityA_analysis['confidence'] - cityA_analysis['consequent support']
cityA_analysis.to_csv('cityA.csv', index=False)

cityA_analysis.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,interest
3296614,"(Café, Hospital, Transit Station, Accountant O...",(Hair Salon),0.060856,0.374814,0.060111,0.987765,2.635348,0.612951
796122,"(Café, Hospital, Clothes Store, Japanese resta...",(Hair Salon),0.063089,0.374814,0.062295,0.987411,2.634405,0.612598
2462421,"(Café, Hospital, Transit Station, Fishing, Lau...",(Hair Salon),0.062196,0.374814,0.061402,0.987231,2.633922,0.612417
4066489,"(Café, Hospital, Transit Station, Japanese res...",(Hair Salon),0.061451,0.374814,0.060657,0.987076,2.633510,0.612262
788125,"(Japanese restaurant, Real Estate, Fishing, La...",(Hair Salon),0.060905,0.374814,0.060111,0.986960,2.633201,0.612146


In [19]:
# Analyse the association rules based on the support value
cityA_analysis.sort_values('support', ascending = False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,interest
1283,(Building Material),(Heavy Industry),0.453291,0.559069,0.335104,0.739269,1.322321,0.180200
1282,(Heavy Industry),(Building Material),0.559069,0.453291,0.335104,0.599396,1.322321,0.146105
1090,(Heavy Industry),(Home Appliances),0.559069,0.458255,0.331927,0.593714,1.295598,0.135459
1091,(Home Appliances),(Heavy Industry),0.458255,0.559069,0.331927,0.724328,1.295598,0.165260
1089,(Building Material),(Home Appliances),0.453291,0.458255,0.301747,0.665681,1.452644,0.207426
1088,(Home Appliances),(Building Material),0.458255,0.453291,0.301747,0.658471,1.452644,0.205180
1059,(Real Estate),(Heavy Industry),0.372679,0.559069,0.292316,0.784363,1.402982,0.225295
1058,(Heavy Industry),(Real Estate),0.559069,0.372679,0.292316,0.522862,1.402982,0.150183
1216,(Heavy Industry),(Hair Salon),0.559069,0.374814,0.286062,0.511675,1.365145,0.136862
1217,(Hair Salon),(Heavy Industry),0.374814,0.559069,0.286062,0.763210,1.365145,0.204141


Range of top 20 support values: 0.335104 to 0.266951	

Observations: 
- All of the rules with the top 20 support values consists of a single antecedent and consequent (X -> Y), which is not surprising.

In [20]:
# Analyse the association rules based on the confidence value
cityA_analysis.sort_values('confidence', ascending = False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,interest
3296614,"(Café, Hospital, Transit Station, Accountant O...",(Hair Salon),0.060856,0.374814,0.060111,0.987765,2.635348,0.612951
796122,"(Café, Hospital, Clothes Store, Japanese resta...",(Hair Salon),0.063089,0.374814,0.062295,0.987411,2.634405,0.612598
2462421,"(Café, Hospital, Transit Station, Fishing, Lau...",(Hair Salon),0.062196,0.374814,0.061402,0.987231,2.633922,0.612417
4066489,"(Café, Hospital, Transit Station, Japanese res...",(Hair Salon),0.061451,0.374814,0.060657,0.987076,2.633510,0.612262
788125,"(Japanese restaurant, Real Estate, Fishing, La...",(Hair Salon),0.060905,0.374814,0.060111,0.986960,2.633201,0.612146
1147600,"(Café, Hospital, Clothes Store, Real Estate, F...",(Hair Salon),0.060806,0.374814,0.060012,0.986939,2.633144,0.612125
2024193,"(Café, Bank, Hospital, Japanese restaurant, La...",(Hair Salon),0.060806,0.374814,0.060012,0.986939,2.633144,0.612125
2043848,"(Café, Chiropractic, Japanese restaurant, Fish...",(Hair Salon),0.060806,0.374814,0.060012,0.986939,2.633144,0.612125
1147972,"(Café, Hospital, Clothes Store, Accountant Off...",(Hair Salon),0.062593,0.374814,0.061749,0.986519,2.632023,0.611705
2365906,"(Bank, Cram School, Japanese restaurant, Real ...",(Hair Salon),0.062295,0.374814,0.061451,0.986454,2.631851,0.611640


Range of top 20 confidence values: 0.987765 to 0.987765

Observations: 
- Based on the top 20 confidence values, all of the rules have (Hair Salon) as the sole consequent.

- As for the antecedents, there were some items that commonly appeared in the top 20 rules:
    - Chiropractic
    - Japanese restaurant
    - Fishing
    - Hospital
    - Café
    - Clothes Store

- By intution, these items should have a strong relation as they are related to health and leisure, which could explain the high frequency of occurance.

In [21]:
# Analyse the association rules based on the lift value
cityA_analysis.sort_values('lift', ascending = False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,interest
4073989,"(Hospital, Transit Station, Japanese restauran...","(Café, Hair Salon, Bank, Heavy Industry, Real ...",0.100665,0.099772,0.060111,0.597140,5.985066,0.497368
4073764,"(Café, Hair Salon, Bank, Heavy Industry, Real ...","(Hospital, Transit Station, Japanese restauran...",0.099772,0.100665,0.060111,0.602488,5.985066,0.501822
4073913,"(Hospital, Transit Station, Café, Accountant O...","(Hair Salon, Bank, Japanese restaurant, Heavy ...",0.097389,0.103594,0.060111,0.617227,5.958151,0.513634
4073840,"(Hair Salon, Bank, Japanese restaurant, Heavy ...","(Hospital, Transit Station, Café, Accountant O...",0.103594,0.097389,0.060111,0.580259,5.958151,0.482870
4109512,"(Hair Salon, Transit Station, Japanese restaur...","(Hospital, Bank, Laundry , Accountant Office)",0.097786,0.104487,0.060558,0.619289,5.926937,0.514802
4109641,"(Hospital, Bank, Laundry , Accountant Office)","(Hair Salon, Transit Station, Japanese restaur...",0.104487,0.097786,0.060558,0.579572,5.926937,0.481786
4073994,"(Real Estate, Hospital, Japanese restaurant, A...","(Café, Hair Salon, Bank, Transit Station, Heav...",0.110196,0.092078,0.060111,0.545495,5.924287,0.453418
4073759,"(Café, Hair Salon, Bank, Transit Station, Heav...","(Real Estate, Hospital, Japanese restaurant, A...",0.092078,0.110196,0.060111,0.652830,5.924287,0.542635
4073769,"(Café, Hair Salon, Hospital, Transit Station, ...","(Real Estate, Bank, Japanese restaurant, Accou...",0.098829,0.102700,0.060111,0.608237,5.922448,0.505537
4073984,"(Real Estate, Bank, Japanese restaurant, Accou...","(Café, Hair Salon, Hospital, Transit Station, ...",0.102700,0.098829,0.060111,0.585307,5.922448,0.486478


Range of top 20 lift values: 5.985066 to 5.913030

Observations: 
- This implies that these groups of items seemingly have quite a significant impact on each other, but to get more information beyond this surface level interpretation would require more detailed analysis as the antecedents and consequents are quite varied.

In [38]:
# Analyse the association rules based on the interest value
cityA_analysis.sort_values('interest', ascending = False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,interest
4109424,"(Hospital, Bank, Transit Station, Accountant O...","(Real Estate, Hair Salon, Laundry )",0.066812,0.197012,0.060558,0.906389,4.600685,0.709377
4105854,"(Hospital, Home Appliances, Transit Station, A...","(Real Estate, Hair Salon, Laundry )",0.067209,0.197012,0.060856,0.905465,4.595995,0.708453
4275226,"(Hospital, Transit Station, Accountant Office,...","(Real Estate, Hair Salon, Laundry )",0.067060,0.197012,0.060657,0.904515,4.591172,0.707503
4110957,"(Hospital, Transit Station, Accountant Office,...","(Real Estate, Hair Salon, Laundry )",0.071379,0.197012,0.064281,0.900556,4.571078,0.703545
4111469,"(Hospital, Transit Station, Accountant Office,...","(Real Estate, Hair Salon, Laundry )",0.072074,0.197012,0.064876,0.900138,4.568953,0.703126
3415301,"(Hospital, Transit Station, Japanese restauran...","(Real Estate, Hair Salon, Laundry )",0.069542,0.197012,0.062543,0.899358,4.564993,0.702346
4152779,"(Hospital, Bank, Accountant Office, Japanese r...","(Real Estate, Hair Salon, Laundry )",0.067855,0.197012,0.060955,0.898317,4.559714,0.701306
4152267,"(Hospital, Bank, Accountant Office, Japanese r...","(Real Estate, Hair Salon, Laundry )",0.067805,0.197012,0.060905,0.898243,4.559336,0.701231
2640721,"(Hair Salon, Hospital, Transit Station, Fishin...","(Real Estate, Bank)",0.068103,0.188822,0.060608,0.889942,4.713135,0.701120
4066568,"(Café, Hospital, Transit Station, Japanese res...","(Real Estate, Hair Salon, Laundry )",0.067557,0.197012,0.060657,0.897869,4.557438,0.700857


Range of top 20 interest values: 0.709377 to 0.697028

Observations: 
- Surprisingly, majority of the consequents consist of the group (Real Estate, Hair Salon, Laundry ). This implies that these items have a meaningful association with many of the items, as the antecedents consists of a mix of different items for the top 20 rules.

## 2. Analysis of City B

In [23]:
cityB_analysis = rules_cityB.iloc[:,0:7]
cityB_analysis['interest'] = cityB_analysis['confidence'] - cityB_analysis['consequent support']
cityB_analysis.to_csv('cityB.csv', index=False)

cityB_analysis.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,interest
393488,"(Home Appliances, Accountant Office, Post Offi...",(Hair Salon),0.030688,0.1895,0.030688,1.0,5.277039,0.8105
393548,"(Heavy Industry, Home Appliances, Post Office,...",(Hair Salon),0.030579,0.1895,0.030579,1.0,5.277039,0.8105
867646,"(Café, Japanese restaurant, Heavy Industry, Re...",(Hair Salon),0.031127,0.1895,0.031127,1.0,5.277039,0.8105
982285,"(Cram School, Interior Shop, Accountant Office...",(Hair Salon),0.030688,0.1895,0.030688,1.0,5.277039,0.8105
1030769,"(Japanese restaurant, Heavy Industry, Real Est...",(Hair Salon),0.031565,0.1895,0.031565,1.0,5.277039,0.8105


In [24]:
# Analyse the association rules based on the support value
cityB_analysis.sort_values('support', ascending = False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,interest
709,(Building Material),(Transit Station),0.270934,0.359711,0.162429,0.599515,1.666658,0.239804
708,(Transit Station),(Building Material),0.359711,0.270934,0.162429,0.451554,1.666658,0.180620
711,(Transit Station),(Heavy Industry),0.359711,0.303924,0.160346,0.445765,1.466700,0.141841
710,(Heavy Industry),(Transit Station),0.303924,0.359711,0.160346,0.527587,1.466700,0.167877
683,(Home Appliances),(Transit Station),0.269838,0.359711,0.153113,0.567425,1.577448,0.207714
682,(Transit Station),(Home Appliances),0.359711,0.269838,0.153113,0.425655,1.577448,0.155817
1042,(Home Appliances),(Building Material),0.269838,0.270934,0.152784,0.566206,2.089833,0.295273
1043,(Building Material),(Home Appliances),0.270934,0.269838,0.152784,0.563916,2.089833,0.294078
1230,(Heavy Industry),(Building Material),0.303924,0.270934,0.151249,0.497656,1.836818,0.226722
1231,(Building Material),(Heavy Industry),0.270934,0.303924,0.151249,0.558252,1.836818,0.254329


Range of top 20 support values: 0.162429 to 0.129220

Observations: 
- The support value in City B is quite low, suggesting that the rules are quite infrequent on a whole.
- As stated before, single antecedent and consequent pair is observed.

In [25]:
# Analyse the association rules based on the confidence value
cityB_analysis.sort_values('confidence', ascending = False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,interest
393488,"(Home Appliances, Accountant Office, Post Offi...",(Hair Salon),0.030688,0.1895,0.030688,1.0,5.277039,0.8105
4781583,"(Hospital, Interior Shop, Accountant Office, T...",(Hair Salon),0.030688,0.1895,0.030688,1.0,5.277039,0.8105
5654581,"(Bank, Hospital, Interior Shop, Accountant Off...",(Hair Salon),0.031565,0.1895,0.031565,1.0,5.277039,0.8105
5566189,"(Hospital, Bank, Interior Shop, Accountant Off...",(Hair Salon),0.032990,0.1895,0.032990,1.0,5.277039,0.8105
5546123,"(Hospital, Bank, Interior Shop, Transit Statio...",(Hair Salon),0.030798,0.1895,0.030798,1.0,5.277039,0.8105
5545615,"(Hospital, Bank, Interior Shop, Accountant Off...",(Hair Salon),0.031017,0.1895,0.031017,1.0,5.277039,0.8105
5533677,"(Hospital, Bank, Park, Interior Shop, Accounta...",(Hair Salon),0.030469,0.1895,0.030469,1.0,5.277039,0.8105
5384579,"(Bank, Hospital, Accountant Office, Real Estat...",(Hair Salon),0.030798,0.1895,0.030798,1.0,5.277039,0.8105
5353591,"(Bank, Hospital, Transit Station, Accountant O...",(Hair Salon),0.030140,0.1895,0.030140,1.0,5.277039,0.8105
5275867,"(Hospital, Bank, Post Office, Japanese restaur...",(Hair Salon),0.030031,0.1895,0.030031,1.0,5.277039,0.8105


Range of top 20 confidence values: 1.0 to 1.0

Observations: 
- Like before, (Hair Salon) is the sole consequent for the top 20 confidence values.
- The confidence is strangely high, indicating that hair salon is always visited for each group of antecedents in the top 20 confidence values. This might be due to the low min_support chosen.

In [26]:
# Bnalyse the association rules based on the lift value
cityB_analysis.sort_values('lift', ascending = False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,interest
7585108,"(Hospital, Interior Shop, Real Estate, Laundry...","(Hair Salon, Bank, Accountant Office, Heavy In...",0.041868,0.043621,0.030579,0.730366,16.743377,0.686745
7584937,"(Hair Salon, Bank, Accountant Office, Heavy In...","(Hospital, Interior Shop, Real Estate, Laundry...",0.043621,0.041868,0.030579,0.701005,16.743377,0.659137
7585095,"(Hospital, Interior Shop, Accountant Office, R...","(Hair Salon, Bank, Heavy Industry, Fishing, Bu...",0.041977,0.043512,0.030579,0.728460,16.741725,0.684948
7584950,"(Hair Salon, Bank, Heavy Industry, Fishing, Bu...","(Hospital, Interior Shop, Accountant Office, R...",0.043512,0.041977,0.030579,0.702771,16.741725,0.660794
7584749,"(Hair Salon, Bank, Accountant Office, Heavy In...","(Real Estate, Hospital, Interior Shop, Laundry )",0.040224,0.045594,0.030579,0.760218,16.673627,0.714624
7585296,"(Real Estate, Hospital, Interior Shop, Laundry )","(Hair Salon, Bank, Accountant Office, Heavy In...",0.045594,0.040224,0.030579,0.670673,16.673627,0.630449
6620141,"(Bank, Fishing, Japanese restaurant, Accountan...","(Hair Salon, Hospital, Interior Shop, Real Est...",0.042087,0.044279,0.030798,0.731771,16.526428,0.687492
6619976,"(Hair Salon, Hospital, Interior Shop, Real Est...","(Bank, Fishing, Japanese restaurant, Accountan...",0.044279,0.042087,0.030798,0.695545,16.526428,0.653458
6619959,"(Hair Salon, Bank, Accountant Office, Japanese...","(Real Estate, Hospital, Interior Shop, Laundry )",0.040881,0.045594,0.030798,0.753351,16.523020,0.707757
6620161,"(Hospital, Fishing, Japanese restaurant, Accou...","(Hair Salon, Bank, Interior Shop, Real Estate,...",0.040881,0.045594,0.030798,0.753351,16.523020,0.707757


Range of top 20 lift values: 16.743377 to 16.377576

Observations: 
- No notable observation

In [27]:
# Analyse the association rules based on the interest value
cityB_analysis.sort_values('interest', ascending = False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,interest
3237741,"(Bank, Post Office, Real Estate, Drug Store, L...","(Hair Salon, Hospital)",0.030907,0.092942,0.030359,0.982270,10.568664,0.889328
5347777,"(Accountant Office, Transit Station, Real Esta...","(Hair Salon, Hospital)",0.030688,0.092942,0.030140,0.982143,10.567301,0.889201
5349300,"(Bank, Accountant Office, Real Estate, Grocery...","(Hair Salon, Hospital)",0.030579,0.092942,0.030031,0.982079,10.566613,0.889137
1372218,"(Drug Store, Bank, Cram School, Laundry )","(Hair Salon, Hospital)",0.032004,0.092942,0.031346,0.979452,10.538350,0.886510
4710745,"(Accountant Office, Japanese restaurant, Heavy...","(Hair Salon, Hospital)",0.031675,0.092942,0.031017,0.979239,10.536055,0.886297
880248,"(Drug Store, Accountant Office, Japanese resta...","(Hair Salon, Hospital)",0.030907,0.092942,0.030250,0.978723,10.530510,0.885782
6550487,"(Bank, Japanese restaurant, Heavy Industry, Re...","(Hair Salon, Hospital)",0.030688,0.092942,0.030031,0.978571,10.528875,0.885630
3167559,"(Accountant Office, Real Estate, Grocery Store...","(Hair Salon, Hospital)",0.032442,0.092942,0.031675,0.976351,10.504988,0.883410
4710999,"(Japanese restaurant, Heavy Industry, Real Est...","(Hair Salon, Hospital)",0.032442,0.092942,0.031675,0.976351,10.504988,0.883410
3167811,"(Heavy Industry, Real Estate, Grocery Store, L...","(Hair Salon, Hospital)",0.032113,0.092942,0.031346,0.976109,10.502383,0.883168


Range of top 20 interest values: 0.889328 to 0.882147

Observations:
- (Hair Salon, Hospital) is the sole consequents group type for the top 20 interest value, indicating a significant relationship with the groups of antecedents.

## 3. Analysis of City C

In [28]:
cityC_analysis = rules_cityC.iloc[:,0:7]
cityC_analysis['interest'] = cityC_analysis['confidence'] - cityC_analysis['consequent support']
cityC_analysis.to_csv('cityC.csv', index=False)

cityC_analysis.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,interest
3951142,"(Elderly Care Home, Bank, Home Appliances, Par...",(Real Estate),0.093817,0.396186,0.093817,1.000000,2.524068,0.603814
4696044,"(Elderly Care Home, Bank, Home Appliances, Par...",(Real Estate),0.092279,0.396186,0.092279,1.000000,2.524068,0.603814
3911771,"(Elderly Care Home, Bank, Hospital, Home Appli...",(Real Estate),0.102122,0.396186,0.101815,0.996988,2.516466,0.600802
2846234,"(Elderly Care Home, Bank, Home Appliances, Acc...",(Real Estate),0.100277,0.396186,0.099969,0.996933,2.516326,0.600747
4628213,"(Elderly Care Home, Bank, Hospital, Home Appli...",(Real Estate),0.100277,0.396186,0.099969,0.996933,2.516326,0.600747


In [29]:
# Analyse the association rules based on the support value
cityC_analysis.sort_values('support', ascending = False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,interest
460,(Transit Station),(Park),0.571516,0.482621,0.392187,0.686222,1.421865,0.203601
461,(Park),(Transit Station),0.482621,0.571516,0.392187,0.812620,1.421865,0.241103
553,(Building Material),(Transit Station),0.472778,0.571516,0.368194,0.778790,1.362673,0.207273
552,(Transit Station),(Building Material),0.571516,0.472778,0.368194,0.644241,1.362673,0.171464
526,(Real Estate),(Transit Station),0.396186,0.571516,0.349431,0.881988,1.543241,0.310471
527,(Transit Station),(Real Estate),0.571516,0.396186,0.349431,0.611410,1.543241,0.215224
502,(Building Material),(Park),0.472778,0.482621,0.338357,0.715680,1.482903,0.233059
503,(Park),(Building Material),0.482621,0.472778,0.338357,0.701083,1.482903,0.228306
528,(Transit Station),(Home Appliances),0.571516,0.434943,0.337742,0.590958,1.358702,0.156015
529,(Home Appliances),(Transit Station),0.434943,0.571516,0.337742,0.776521,1.358702,0.205004


Range of top 20 support values: 0.392187 to 0.318056

Observations:
- No notable observation

In [30]:
# Analyse the association rules based on the confidence value
cityC_analysis.sort_values('confidence', ascending = False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,interest
3951142,"(Elderly Care Home, Bank, Home Appliances, Par...",(Real Estate),0.093817,0.396186,0.093817,1.000000,2.524068,0.603814
4696044,"(Elderly Care Home, Bank, Home Appliances, Par...",(Real Estate),0.092279,0.396186,0.092279,1.000000,2.524068,0.603814
3911771,"(Elderly Care Home, Bank, Hospital, Home Appli...",(Real Estate),0.102122,0.396186,0.101815,0.996988,2.516466,0.600802
4628213,"(Elderly Care Home, Bank, Hospital, Home Appli...",(Real Estate),0.100277,0.396186,0.099969,0.996933,2.516326,0.600747
2846234,"(Elderly Care Home, Bank, Home Appliances, Acc...",(Real Estate),0.100277,0.396186,0.099969,0.996933,2.516326,0.600747
4098970,"(Elderly Care Home, Bank, Home Appliances, Tra...",(Real Estate),0.098739,0.396186,0.098431,0.996885,2.516205,0.600699
4749593,"(Elderly Care Home, Hair Salon, Hospital, Bank...",(Real Estate),0.097201,0.396186,0.096893,0.996835,2.516081,0.600650
3285154,"(Elderly Care Home, Bank, Home Appliances, Par...",(Real Estate),0.096278,0.396186,0.095970,0.996805,2.516004,0.600619
4751124,"(Elderly Care Home, Bank, Hospital, Home Appli...",(Real Estate),0.096278,0.396186,0.095970,0.996805,2.516004,0.600619
3951397,"(Elderly Care Home, Bank, Home Appliances, Par...",(Real Estate),0.095970,0.396186,0.095663,0.996795,2.515978,0.600609


Range of top 20 confidence values: 1.000000 to 0.996689

Observations:
- The sole consequent is (Real Estate) for the top 20 confidence values
- The group of items in antecedents are quite similar to one another.

In [31]:
# Analyse the association rules based on the lift value
cityC_analysis.sort_values('lift', ascending = False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,interest
4282590,"(Building Material, Hair Salon, Interior Shop,...","(Hospital, Bank, Accountant Office, Heavy Indu...",0.121501,0.142725,0.090126,0.741772,5.197201,0.599047
4282783,"(Hospital, Bank, Accountant Office, Heavy Indu...","(Building Material, Hair Salon, Interior Shop,...",0.142725,0.121501,0.090126,0.631466,5.197201,0.509964
2877828,"(Hospital, Bank, Interior Shop, Accountant Off...","(Real Estate, Transit Station, Japanese restau...",0.134420,0.129499,0.090126,0.670481,5.177511,0.540982
2877897,"(Real Estate, Transit Station, Japanese restau...","(Hospital, Bank, Interior Shop, Accountant Off...",0.129499,0.134420,0.090126,0.695962,5.177511,0.561542
2877953,"(Real Estate, Japanese restaurant, Convenience...","(Hospital, Bank, Interior Shop, Accountant Off...",0.132882,0.131037,0.090126,0.678241,5.175964,0.547204
2877772,"(Hospital, Bank, Interior Shop, Accountant Off...","(Real Estate, Japanese restaurant, Convenience...",0.131037,0.132882,0.090126,0.687793,5.175964,0.554911
4200523,"(Hair Salon, Hospital, Bank, Accountant Office...","(Elderly Care Home, Transit Station, Japanese ...",0.155337,0.113196,0.090741,0.584158,5.160595,0.470962
4200630,"(Elderly Care Home, Transit Station, Japanese ...","(Hair Salon, Hospital, Bank, Accountant Office...",0.113196,0.155337,0.090741,0.801630,5.160595,0.646294
4282595,"(Building Material, Hair Salon, Heavy Industry...","(Hospital, Bank, Interior Shop, Accountant Off...",0.130114,0.134420,0.090126,0.692671,5.153031,0.558251
4282778,"(Hospital, Bank, Interior Shop, Accountant Off...","(Building Material, Hair Salon, Heavy Industry...",0.134420,0.130114,0.090126,0.670481,5.153031,0.540367


Range of top 20 lift values: 5.197201 to 5.130119

Observations:
- No notable observation

In [32]:
# Analyse the association rules based on the interest value
cityC_analysis.sort_values('interest', ascending = False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,interest
2887963,"(Elderly Care Home, Transit Station, Accountan...","(Hospital, Bank, Real Estate)",0.100892,0.200246,0.093817,0.929878,4.643677,0.729632
2896811,"(Elderly Care Home, Transit Station, Accountan...","(Real Estate, Bank)",0.093202,0.249154,0.091049,0.976898,3.920857,0.727744
1501101,"(Elderly Care Home, Accountant Office, Japanes...","(Bank, Hospital, Real Estate)",0.102430,0.200246,0.095048,0.927928,4.633938,0.727682
2896854,"(Elderly Care Home, Accountant Office, Japanes...","(Real Estate, Bank, Transit Station)",0.094432,0.236543,0.091049,0.964169,4.076092,0.727627
4282458,"(Hair Salon, Hospital, Interior Shop, Accounta...","(Real Estate, Bank)",0.092279,0.249154,0.090126,0.976667,3.919930,0.727513
4200398,"(Elderly Care Home, Hair Salon, Accountant Off...","(Hospital, Bank, Real Estate)",0.097816,0.200246,0.090741,0.927673,4.632665,0.727427
4267727,"(Hair Salon, Hospital, Interior Shop, Accounta...","(Real Estate, Bank, Transit Station)",0.099046,0.236543,0.095355,0.962733,4.070019,0.726190
2903443,"(Elderly Care Home, Hair Salon, Accountant Off...","(Hospital, Bank, Real Estate)",0.099354,0.200246,0.091972,0.925697,4.622795,0.725451
4200418,"(Elderly Care Home, Hospital, Transit Station,...","(Real Estate, Hair Salon, Bank)",0.096586,0.214088,0.090741,0.939490,4.388338,0.725402
2905257,"(Hospital, Accountant Office, Japanese restaur...","(Real Estate, Hair Salon, Bank)",0.096278,0.214088,0.090434,0.939297,4.387435,0.725209


Range of top 20 interest values: 0.729632 to 0.724268

Observations:
- (Elderly Care Home) appears very frequently in the antecedents group, perhaps suggesting a high population of elderly or their family members.

## 4. Analysis of City D

In [33]:
cityD_analysis = rules_cityD.iloc[:,0:7]
cityD_analysis['interest'] = cityD_analysis['confidence'] - cityD_analysis['consequent support']
cityD_analysis.to_csv('cityD.csv', index=False)

cityD_analysis.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,interest
711730,"(Hospital, Transit Station, Accountant Office,...",(Hair Salon),0.032490,0.230251,0.032308,0.994398,4.318752,0.764147
941177,"(Hospital, Transit Station, Accountant Office,...",(Hair Salon),0.030943,0.230251,0.030761,0.994118,4.317535,0.763866
941685,"(Hospital, Transit Station, Accountant Office,...",(Hair Salon),0.031216,0.230251,0.030943,0.991254,4.305097,0.761002
943463,"(Hospital, Transit Station, Accountant Office,...",(Hair Salon),0.030397,0.230251,0.030124,0.991018,4.304073,0.760767
713368,"(Hospital, Transit Station, Accountant Office,...",(Hair Salon),0.032763,0.230251,0.032399,0.988889,4.294827,0.758638


In [34]:
# Analyse the association rules based on the support value
cityD_analysis.sort_values('support', ascending = False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,interest
1138,(Heavy Industry),(Building Material),0.334274,0.322625,0.172370,0.515655,1.598312,0.193030
1139,(Building Material),(Heavy Industry),0.322625,0.334274,0.172370,0.534274,1.598312,0.200000
948,(Home Appliances),(Building Material),0.331361,0.322625,0.171460,0.517440,1.603846,0.194816
949,(Building Material),(Home Appliances),0.322625,0.331361,0.171460,0.531453,1.603846,0.200091
952,(Heavy Industry),(Home Appliances),0.334274,0.331361,0.162268,0.485434,1.464969,0.154073
953,(Home Appliances),(Heavy Industry),0.331361,0.334274,0.162268,0.489701,1.464969,0.155427
643,(Transit Station),(Heavy Industry),0.308063,0.334274,0.155442,0.504579,1.509478,0.170305
642,(Heavy Industry),(Transit Station),0.334274,0.308063,0.155442,0.465015,1.509478,0.156952
638,(Transit Station),(Building Material),0.308063,0.322625,0.154350,0.501034,1.552993,0.178409
639,(Building Material),(Transit Station),0.322625,0.308063,0.154350,0.478420,1.552993,0.170357


Range of top 20 support values: 0.172370 to 0.142064

Observations:
- No notable observations

In [35]:
# Analyse the association rules based on the confidence value
cityD_analysis.sort_values('confidence', ascending = False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,interest
711730,"(Hospital, Transit Station, Accountant Office,...",(Hair Salon),0.032490,0.230251,0.032308,0.994398,4.318752,0.764147
941177,"(Hospital, Transit Station, Accountant Office,...",(Hair Salon),0.030943,0.230251,0.030761,0.994118,4.317535,0.763866
941685,"(Hospital, Transit Station, Accountant Office,...",(Hair Salon),0.031216,0.230251,0.030943,0.991254,4.305097,0.761002
943463,"(Hospital, Transit Station, Accountant Office,...",(Hair Salon),0.030397,0.230251,0.030124,0.991018,4.304073,0.760767
713368,"(Hospital, Transit Station, Accountant Office,...",(Hair Salon),0.032763,0.230251,0.032399,0.988889,4.294827,0.758638
723826,"(Hospital, Transit Station, Accountant Office,...",(Hair Salon),0.031580,0.230251,0.031216,0.988473,4.293019,0.758221
712990,"(Hospital, Bank, Transit Station, Accountant O...",(Hair Salon),0.031398,0.230251,0.031034,0.988406,4.292728,0.758155
967339,"(Hospital, Bank, Accountant Office, Heavy Indu...",(Hair Salon),0.031216,0.230251,0.030852,0.988338,4.292435,0.758087
723952,"(Hospital, Transit Station, Japanese restauran...",(Hair Salon),0.031034,0.230251,0.030670,0.988270,4.292138,0.758019
941431,"(Hospital, Transit Station, Accountant Office,...",(Hair Salon),0.030397,0.230251,0.030033,0.988024,4.291070,0.757773


Range of top 20 confidence values: 0.994398 to 0.985714

Observations:
- (Hair Salon) is once again the sole consequent
- The antecedent group is also very similar to one another.

In [36]:
# Analyse the association rules based on the lift value
cityD_analysis.sort_values('lift', ascending = False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,interest
943565,"(Fishing, Hair Salon, Accountant Office, Trans...","(Real Estate, Hospital, Laundry , Japanese res...",0.049691,0.048689,0.030124,0.606227,12.450885,0.557538
943600,"(Real Estate, Hospital, Laundry , Japanese res...","(Fishing, Hair Salon, Accountant Office, Trans...",0.048689,0.049691,0.030124,0.618692,12.450885,0.569001
1005583,"(Hair Salon, Bank, Transit Station, Heavy Indu...","(Hospital, Japanese restaurant, Accountant Off...",0.051147,0.050055,0.031398,0.613879,12.264186,0.563824
1005794,"(Hospital, Japanese restaurant, Accountant Off...","(Hair Salon, Bank, Transit Station, Heavy Indu...",0.050055,0.051147,0.031398,0.627273,12.264186,0.576126
1005723,"(Hair Salon, Hospital, Laundry , Japanese rest...","(Bank, Transit Station, Accountant Office, Hea...",0.053695,0.047688,0.031398,0.584746,12.261806,0.537057
1005654,"(Bank, Transit Station, Accountant Office, Hea...","(Hair Salon, Hospital, Laundry , Japanese rest...",0.047688,0.053695,0.031398,0.658397,12.261806,0.604702
1005069,"(Hair Salon, Bank, Transit Station, Accountant...","(Hospital, Laundry , Japanese restaurant, Buil...",0.051511,0.048963,0.030761,0.597173,12.196540,0.548211
1005288,"(Hospital, Laundry , Japanese restaurant, Buil...","(Hair Salon, Bank, Transit Station, Accountant...",0.048963,0.051511,0.030761,0.628253,12.196540,0.576742
1005580,"(Hair Salon, Bank, Transit Station, Accountant...","(Real Estate, Hospital, Laundry , Japanese res...",0.052967,0.048689,0.031398,0.592784,12.174776,0.544094
1005797,"(Real Estate, Hospital, Laundry , Japanese res...","(Hair Salon, Bank, Transit Station, Accountant...",0.048689,0.052967,0.031398,0.644860,12.174776,0.591893


Range of top 20 lift values: 12.450885 to 12.100060

Observations:
- No notable observations.

In [37]:
# Analyse the association rules based on the interest value
cityD_analysis.sort_values('interest', ascending = False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,interest
943485,"(Hospital, Transit Station, Accountant Office,...","(Hair Salon, Laundry )",0.031580,0.124317,0.030124,0.953890,7.673022,0.829573
773721,"(Hair Salon, Bank, Transit Station, Accountant...",(Hospital),0.032126,0.143975,0.031125,0.968839,6.729202,0.824863
1005471,"(Hospital, Bank, Transit Station, Accountant O...","(Hair Salon, Laundry )",0.033127,0.124317,0.031398,0.947802,7.624049,0.823485
450478,"(Hair Salon, Bank, Transit Station, Real Estat...",(Hospital),0.032854,0.143975,0.031762,0.966759,6.714758,0.822784
164048,"(Real Estate, Grocery Store, Drug Store)","(Hair Salon, Hospital)",0.033127,0.105934,0.030761,0.928571,8.765587,0.822638
454074,"(Hair Salon, Bank, Accountant Office, Real Est...",(Hospital),0.032581,0.143975,0.031489,0.966480,6.712824,0.822505
941199,"(Hospital, Transit Station, Accountant Office,...","(Hair Salon, Laundry )",0.032490,0.124317,0.030761,0.946779,7.615816,0.822461
451780,"(Hair Salon, Bank, Transit Station, Accountant...",(Hospital),0.034037,0.143975,0.032854,0.965241,6.704212,0.821265
773846,"(Hair Salon, Bank, Transit Station, Heavy Indu...",(Hospital),0.031125,0.143975,0.030033,0.964912,6.701932,0.820937
454322,"(Hair Salon, Accountant Office, Heavy Industry...",(Hospital),0.033309,0.143975,0.032126,0.964481,6.698935,0.820506


Range of top 20 interest values: 0.829573 to 0.818124

Observations:
- (Hair Salon), (Laundry) and (Hospital) are the sole consequents